In [ ]:
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion
import astropy.units as u
import os
from pathlib import Path
from astropy import units as u
import numpy as np
import glob
import matplotlib.pyplot as plt

from gammapy.data import DataStore

from gammapy.datasets import (
    Datasets,
    SpectrumDataset,
)

from gammapy.estimators import FluxPointsEstimator
from gammapy.makers import (
    ReflectedRegionsBackgroundMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
)

from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    LogParabolaSpectralModel,
    PowerLawSpectralModel,
    SkyModel,
)

In [ ]:
# This is directory with IRFs which were used to produce DL3 files
irf_dir = ".../IRFs/"

# This is directory containing hdu-index.fits.gz and obs-index.fits.gz. These can be either for one particular 
# night (as authomaticaly produced in Daily analysis), or for any other time interval. One can use 
# https://github.com/SST-1M-collaboration/sst1mpipe/blob/main/sst1mpipe/scripts/create_hdu_indexes.py
# script to produce joint HDU indexes for some bunch of data to be analyzed.
datastore = ""

# Here you should specify some coordinates where you expect to see something interesting.
# Either this way:
target_position = SkyCoord.from_name("source name")

# Or this way:
# source_pos = SkyCoord(10., -25, frame="icrs", unit="deg")
# See https://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html

In [ ]:
os.environ['CALDB'] = irf_dir

# Define the target and on region

In [ ]:
on_region_radius = Angle("0.2 deg")
on_region = CircleSkyRegion(center=target_position, radius=on_region_radius)

# Load the Data

In [ ]:
## load datastore
data_store = DataStore.from_dir(datastore)
data_store.obs_table.sort('TSTART')

In [ ]:
# We can remove some obsids (i.e. wobbles) we do not like
# These particular ones are just examples
bad_obsids = np.array([202309140409, 202309170461, 202310110480])
obsid_mask = [obsid not in list(bad_obsids.astype(int)) for obsid in data_store.obs_table["OBS_ID"]]
good_obs_list = data_store.obs_table[obsid_mask]['OBS_ID']

In [ ]:
data_store.obs_table[obsid_mask]

In [ ]:
selected_obs_table = data_store.obs_table[obsid_mask]

In [ ]:
observations = data_store.get_observations(good_obs_list)

In [ ]:
# optional : we define exclusion region around the target to avoid contamination in the OFF regions

exclusion_region = CircleSkyRegion(
    center=target_position,
    radius=0.25 * u.deg,
)
geom = WcsGeom.create(
    npix=(150, 150), binsz=0.05, skydir=target_position, proj="TAN", frame="icrs"
)

exclusion_mask = ~geom.region_mask([exclusion_region])
exclusion_mask.plot()
plt.show()

In [ ]:
# reco energy axis should be the same also for the flux points
emin=1
emax=100
nbin = 12
energy_axis = MapAxis.from_energy_bounds(
    emin, emax, nbin=nbin, per_decade=True, unit="TeV", name="energy"
)

energy_axis_true = MapAxis.from_energy_bounds(
    0.2, 150, nbin=41, per_decade=True, unit="TeV", name="energy_true"
)

geom = RegionGeom.create(region=on_region, axes=[energy_axis])
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

dataset_maker = SpectrumDatasetMaker(
    containment_correction=True, selection=["counts", "exposure", "edisp"]
)

bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)
safe_mask_masker = SafeMaskMaker(methods=["aeff-max", "edisp-bias"], aeff_percent=1, bias_percent=30)

In [ ]:
unstacked_datasets = Datasets()

## loop on the observation : 
for obs_id, observation in zip(selected_obs_table["OBS_ID"], observations):
    print("processing obs {}".format(obs_id))
    dataset = dataset_maker.run(dataset_empty.copy(name=str(obs_id)), observation)
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    unstacked_datasets.append(dataset_on_off)

# Some cumulative stats

In [ ]:
info_table = unstacked_datasets.info_table(cumulative=True)
info_table

In [ ]:
fig, (ax_excess, ax_sqrt_ts) = plt.subplots(figsize=(10, 4), ncols=2, nrows=1)
ax_excess.plot(
    info_table["livetime"].to("h"),
    info_table["excess"],
    marker=".",
    ls="none",
)

ax_excess.set_title("Excess")
ax_excess.set_xlabel("Livetime [h]")
ax_excess.set_ylabel("Excess events")

ax_sqrt_ts.plot(
    info_table["livetime"].to("h"),
    info_table["sqrt_ts"],
    marker=".",
    ls="none",
)

ax_sqrt_ts.set_title("Sqrt(TS)")
ax_sqrt_ts.set_xlabel("Livetime [h]")
ax_sqrt_ts.set_ylabel("Sqrt(TS)")

ax_excess.grid()
ax_sqrt_ts.grid()
plt.show()

# Define spectral models

In [ ]:
## Log parabola model : 
logp_model = LogParabolaSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    reference = 5. * u.TeV,
    )

## PL
pwl_model = PowerLawSpectralModel(
    amplitude=1e-12 * u.Unit("cm-2 s-1 TeV-1"),
    index=2,
    reference=5. * u.TeV,
)

# Fit of stacked dataset

In [ ]:
dataset_stacked = Datasets(unstacked_datasets).stack_reduce(name='Crab')

In [ ]:
# For likelihood ratio test (see https://github.com/luca-giunti/CTAO-CTAC_Meeting_Bologna_2022/blob/main/1D_analysis.ipynb)

# H0 hypothesis - there is only background in the data
Wstat_0 = dataset_stacked.stat_sum()
# this is -2 logL for H0 hypothesis
print("-2 logL for H0 hypothesis:", Wstat_0)
# residual with H0 hypothesis, so if there is a source, there should be positive residuals suggesting that we miss something in the model (the source)
dataset_stacked.plot_residuals_spectral()

In [ ]:
## define the source model
model = SkyModel(spectral_model=pwl_model, name="source")

dataset_stacked.models = [model]

# run the FIT !
fit = Fit()
result_stacked = fit.run(datasets=dataset_stacked)

model_best_stacked = model.copy()

In [ ]:
## check if the fit converged
print(result_stacked)

In [ ]:
# Fit model covariance matrix plot
model_best_stacked.covariance.plot_correlation()

In [ ]:
display(result_stacked.models.to_parameters_table())

# Fit quality and model residuals


In [ ]:
ax_spectrum, ax_residuals = dataset_stacked.plot_fit()
dataset_stacked.plot_masks(ax=ax_spectrum)
plt.show()

In [ ]:
print(dataset_stacked)

In [ ]:
from scipy.stats import chi2

# For H1 hypothesis - there is a source with power-law spectrum
Wstat_1 = result_stacked.total_stat
print("-2 logL for H1 hypothesis:", Wstat_1)

# dof for PL is 2 (sp index and normalization)
dof = 2
# this number is significance of detection
print("delta TS of detection: ", (Wstat_0-Wstat_1), "p-value: ", chi2.sf((Wstat_0-Wstat_1), dof))

# It can be then converted to sigma for PL model with 2 deg of freedom:

# TS -> sigma prescription is from gammapy documentation
# https://docs.gammapy.org/0.18.2/stats/index.html

from scipy.stats import norm
p_value = chi2.sf((Wstat_0-Wstat_1), dof)
sigma = norm.isf(p_value / 2)
print("significance in units of sigma:", sigma)
# test that we get the same p_value from this sigma:
p_value = 2 * (1 - norm.cdf(sigma))
print(p_value)

# Compute Flux Point
To round up our analysis we can compute flux points by fitting the norm of the global model in energy bands. We can utilise the resample_energy_edges for defining the energy bins in which the minimum number of sqrt_ts is 2.

In [ ]:
energy_edges = np.geomspace(emin, emax, nbin+1) * u.TeV

In [ ]:
fpe = FluxPointsEstimator(
    energy_edges=energy_edges, source="source", selection_optional="all"
)
flux_points = fpe.run(datasets=dataset_stacked)

In [ ]:
display(flux_points.to_table(sed_type="dnde", formatted=True))

In [ ]:
fig, ax = plt.subplots()
ax.yaxis.set_units(u.TeV/(u.cm**2 * u.s))
flux_points.plot(ax=ax, sed_type="e2dnde", color="darkorange")
flux_points.plot_ts_profiles(ax=ax, sed_type="e2dnde")

plt.show()
